In [1]:
!pip install zarr

In [2]:
import os
from pathlib import Path
import imageio

data_dir = Path("./data/example_images")

file_name = "0_0"
channels = [
    "_0_fluorescence.bmp"
    # "_0_BF_LED_matrix_full.jpg",
    # "_0_BF_LED_matrix_left_half.jpg",
    # "_0_BF_LED_matrix_right_half.jpg",
    # "_0_Fluorescence_405_nm_Ex.jpg",
]


# image_bf = imageio.imread(data_dir / f"{file_name}{channels[1]}")
# image_DAPI = imageio.imread(data_dir / f"{file_name}{channels[3]}")
# image_DAPI.shape

In [3]:
import gcsfs
import cv2
import time

# gcsfs
print('------')
fs = gcsfs.GCSFileSystem(project='soe-octopi',token='whole-slide-20220214-keys.json')
# list datasets
for folder in fs.ls('octopi-malaria-whole-slide/zarr-test'):
	print(folder)
    # pass
print('------')
data_dir2 = 'octopi-malaria-tanzania-2021-data/U3D_201910_2022-01-11_23-11-36.799392/0/'

# access individual files
filename = 'octopi-malaria-tanzania-2021-data/U3D_201910_2022-01-11_23-11-36.799392/0/0_0_0_Fluorescence_405_nm_Ex.bmp'
# print(fs.ls(filename))
img_bytes = fs.cat(filename)
image_bf = imageio.core.asarray(imageio.imread(img_bytes, "bmp"))
image_bf.shape

------
octopi-malaria-whole-slide/zarr-test/store
octopi-malaria-whole-slide/zarr-test/test2
octopi-malaria-whole-slide/zarr-test/test3fluorescence_405_5x5
octopi-malaria-whole-slide/zarr-test/test4
octopi-malaria-whole-slide/zarr-test/test5
------


(3000, 3000, 3)

In [4]:
import zarr
from imjoy_rpc import api
from imjoy_rpc import register_default_codecs

register_default_codecs()

class Plugin:
    def __init__(self, images, view_state=None):
        if not isinstance(images, list):
            images = [images]
        self.images = images
        self.view_state = view_state

    async def setup(self):
        pass

    async def run(self, ctx):
        viewer = await api.createWindow(
            type="vizarr", src="https://hms-dbmi.github.io/vizarr"
        )
        if self.view_state:
            await viewer.set_view_state(self.view_state)
        for img in self.images:
            await viewer.add_image(img)


def run_vizarr(images, view_state=None):
    api.export(Plugin(images, view_state))


In [5]:
import zarr
import numpy as np
import os
from skimage import data
from skimage.transform import pyramid_gaussian

# Modified from https://github.com/ome/ome-zarr-py/blob/master/tests/create_test_data.py
def create_ome_zarr(img_path, img_name, dims, zarr_directory, n_layers, fs, dtype="f4", gs = False): # gs = greyscale t/f, full image path = img_path + x_ + y_ + img_name
    # store = fs.get_mapper('gs://octopi-malaria-whole-slide/zarr-test/' + zarr_directory)
    store = zarr.DirectoryStore(zarr_directory)
    # store = zarr.ZipStore(zarr_directory)
    grp = zarr.group(store = store, overwrite=True)
    paths = []
    for path in range(n_layers+1):
        grp.create_dataset(str(path), shape=(1, 3, 1, (3000/2**path)*dims[0], (3000/2**path)*dims[1]), dtype='f4')
        paths.append({"path": str(path)})
    print(grp.tree())
    
    for x in range(dims[0]):
        for y in range(dims[1]):
            print(x, y)
            # image = imageio.imread(img_path / f"{y}_{x}{img_name}")
            img_bytes = fs.cat(img_path + f"{y}_{x}{img_name}")
            image = imageio.core.asarray(imageio.imread(img_bytes, "bmp"))
            # assert image.ndim == 1
            base = image #np.tile(image, (4, 4, 1))
            if gs:
                gaussian = list(pyramid_gaussian(base, downscale=2, max_layer=n_layers, channel_axis=None))
            else:
                gaussian = list(pyramid_gaussian(base, downscale=2, max_layer=n_layers, channel_axis=-1))
            print(image.shape)
            for path, pixels in enumerate(gaussian):
                dim = int(np.around(3000/2**path))
                # print(dim*(dims[1]-y-1), dim*(dims[1]-y)+1)
                # print( dim*(x), dim*(x+1))
                # print(pixels[:, :, 0].shape)
                # print(pixels.shape)
                if gs: # if statment for greyscale or not
                    grp[str(path)][0,0,0, dim*(dims[1]-y-1):dim*(dims[1]-y), dim*(x):dim*(x+1)] = pixels
                else:
                    grp[str(path)][0,0,0, dim*(dims[1]-y-1):dim*(dims[1]-y), dim*(x):dim*(x+1)] = pixels[:,:,0]
                    grp[str(path)][0,1,0, dim*(dims[1]-y-1):dim*(dims[1]-y), dim*(x):dim*(x+1)] = pixels[:,:,1]
                    grp[str(path)][0,2,0, dim*(dims[1]-y-1):dim*(dims[1]-y), dim*(x):dim*(x+1)] = pixels[:,:,2]
            # store.flush()
            
    if gs: #different metadate for bright field or flourecense
        image_data = {
            "id": 1,
            "channels": [
                {
                    "color": "FFFFFF",
                    "window": {"start": 0, "end": 1},
                    "label": "Brightness",
                    "active": True,
                }
            ],
            "rdefs": {
                "model": "greyscale",
            },
        }
        
    else:
    
        image_data = {
            "id": 1,
            "channels": [
                {
                    "color": "FF0000",
                    "window": {"start": 0, "end": 1},
                    "label": "Red",
                    "active": True,
                },
                {
                    "color": "00FF00",
                    "window": {"start": 0, "end": 1},
                    "label": "Green",
                    "active": True,
                },
                {
                    "color": "0000FF",
                    "window": {"start": 0, "end": 1},
                    "label": "Blue",
                    "active": True,
                },
            ],
            "rdefs": {
                "model": "color",
            },
        }

    multiscales = [
        {
            "version": "0.1",
            "datasets": paths,
        }
    ]
    grp.attrs["multiscales"] = multiscales
    grp.attrs["omero"] = image_data
    # store.close()


In [6]:
# Save it to /tmp/astronaut.zarr
# create_ome_zarr(image_bf, "./bf.zarr")
# create_ome_zarr(image_DAPI, "./DAPI.zarr")
# create_ome_zarr(data_dir, channels[0], (2,2), "./fluorescence.zarr", 3)
create_ome_zarr(data_dir2, '_0_Fluorescence_405_nm_Ex.bmp', (2,2), "./fluorescence_405_2x2.zarr", 3, fs)
create_ome_zarr(data_dir2, '_0_BF_LED_matrix_low_NA.bmp', (2,2), "./bf_low_2x2.zarr", 3, fs, gs = True)

/
 ├── 0 (1, 3, 1, 6000, 6000) float32
 ├── 1 (1, 3, 1, 3000, 3000) float32
 ├── 2 (1, 3, 1, 1500, 1500) float32
 └── 3 (1, 3, 1, 750, 750) float32
0 0
(3000, 3000, 3)
0 1
(3000, 3000, 3)
1 0
(3000, 3000, 3)
1 1
(3000, 3000, 3)
/
 ├── 0 (1, 3, 1, 6000, 6000) float32
 ├── 1 (1, 3, 1, 3000, 3000) float32
 ├── 2 (1, 3, 1, 1500, 1500) float32
 └── 3 (1, 3, 1, 750, 750) float32
0 0
(3000, 3000)
0 1
(3000, 3000)
1 0
(3000, 3000)
1 1
(3000, 3000)


In [7]:
import zarr
zarr_image_fl = zarr.open("./fluorescence_405_2x2.zarr", mode="r")
zarr_image_bf = zarr.open("./bf_low_2x2.zarr", mode="r")

zarr_fl = { "source": zarr_image_fl, "name": "fluorescence" }
zarr_bf = { "source": zarr_image_bf, "name": "bright field" }

# Run vizarr
run_vizarr([zarr_bf,zarr_fl])
# run_vizarr([zarr_fl])

<IPython.core.display.Javascript object>